# Packages

In [1]:
# Data analysis and Excel reading
import pandas as pd

#
from dotenv import load_dotenv

# File and directory operations
import os

# Send HTTP requests (e.g., to Google Street View API)
import requests

# Image processing (OpenCV)
import cv2

# Display images and HTML in Jupyter notebooks
from IPython.display import Image, display, HTML

# Excel file manipulation and styling
from openpyxl import load_workbook
from openpyxl.styles import Alignment

# Download Street View images

Google Street View images were automatically downloaded using the Street View Static API for multiple georeferenced locations, based on latitude and longitude coordinates extracted from an Excel file.

For each central point of the selected street segment, images were captured in eight different directions, spaced every 30 degrees to cover the full 360-degree view around the location. The images were obtained with a standard resolution of 640x640 pixels, a field of view of 60 degrees, and a fixed pitch angle of zero.

All images were stored locally in folders organized by location ID.

## Parameters

In [10]:
# Authenticate requests to the Google API

load_dotenv("api_key.env")
api_key = os.getenv("GOOGLE_API_KEY")

In [ ]:
# Image size (width x height)
size = "640x640"

# Camera pitch (vertical angle)
pitch = 0

# Field of view (horizontal angle)
fov = 60

# Headings (camera directions in degrees)
# List of headings every 30 degrees around 360°
headings = [i for i in range(0, 360, 30)]

## Download function

In [ ]:
def download_images(location_id, lat, lon, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    base_url = "https://maps.googleapis.com/maps/api/streetview"
    image_paths = []
    success = False

    for heading in headings:
        filename = os.path.join(output_folder, f"{location_id}_h{heading}.jpg")

        if not os.path.exists(filename):
            params = {
                "key": api_key,
                "size": size,
                "pitch": pitch,
                "fov": fov,
                "heading": heading,
                "location": f"{lat},{lon}"
            }
            response = requests.get(base_url, params=params)

            if response.status_code == 200:
                with open(filename, "wb") as f:
                    f.write(response.content)
                image_paths.append(filename)
                success = True
            elif response.status_code != 404:
                print(f"❌ Error {response.status_code} at ID {location_id} | Heading: {heading}")
        else:
            image_paths.append(filename)
            success = True

    return success, sorted(list(set(image_paths)))

output_base_folder = "street_view_images"

excel_file = "coordinates.xlsx"
df = pd.read_excel(excel_file)

coordinates = list(zip(df['id'], df['latitude'], df['longitude']))

for location_id, lat, lon in coordinates:
    folder = os.path.join(output_base_folder, str(location_id))
    success, _ = download_images(location_id, lat, lon, folder)
    if success:
        print(f"✅ Downloading completed for ID: {location_id}")
    else:
        print(f"⚠️ No images found or all requests failed for ID: {location_id}")

# Stitching

After the download, the images obtained for each location were processed using a stitching algorithm to generate a 360-degree panoramic view of the surroundings from the central point. The individual images, captured in eight distinct directions, were loaded and automatically combined using the OpenCV library.

The stitching function assessed the overlap between images and performed alignment and blending to produce a seamless composite image. When successful, the resulting panoramic image was cropped to remove unwanted margins and then saved locally under a filename corresponding to the location's unique ID. The stitched panoramas were stored in the same ID-organized folders used to save the original images.

In [ ]:
def load_images(image_paths):
    images = []
    for path in image_paths:
        img = cv2.imread(path)
        if img is not None:
            images.append(img)
    return images

def stitch_and_save(images, output_path, location_id):
    if not images:
        print(f"❌ No valid images to stitch for ID: {location_id}")
        return

    stitcher = getattr(cv2, "Stitcher_create", lambda: cv2.Stitcher.create(False))()
    status, stitched = stitcher.stitch(images)

    if status == 0:
        margin = 15
        h, w = stitched.shape[:2]
        stitched_cropped = stitched[margin:h - margin, margin:w - margin]
        cv2.imwrite(output_path, stitched_cropped)
        print(f"✅ Stitching completed for ID: {location_id}")
    else:
        errors = {
            -1: "More images or better overlap needed.",
            -2: "Global alignment failure.",
            -3: "Homography estimation failure.",
            -4: "Camera parameter adjustment failure."
        }
        print(f"❌ Stitching failed for ID: {location_id} → {errors.get(status, f'Error {status}')}")

for location_id, lat, lon in coordinates:
    folder = os.path.join(output_base_folder, str(location_id))
    stitched_file = os.path.join(folder, f"{location_id}.jpg")

    if os.path.exists(stitched_file):
        print(f"✅ Stitching completed for ID: {location_id}")
        continue

    image_paths = [os.path.join(folder, f) for f in os.listdir(folder) if f.endswith(".jpg")]

    imgs = load_images(image_paths)
    stitch_and_save(imgs, stitched_file, location_id)

# Get image date via the metadata API

An automated extraction of image dates from Google Street View was performed using the Street View Metadata API, based on geographic coordinates retrieved from an Excel file. For each point, the date of the most recent available image was recorded. The results were stored in a new Excel file containing the fields: id, latitude, longitude, and date.

In [ ]:
def get_metadata(lat, lon):
    url = "https://maps.googleapis.com/maps/api/streetview/metadata"
    params = {
        "location": f"{lat},{lon}",
        "key": api_key
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        return data.get("date", "Unknown")
    else:
        return "Error"

excel_file = "coordinates.xlsx"
df = pd.read_excel(excel_file)

output_file = "street_view_dates.xlsx"

if os.path.exists(output_file):
    existing_df = pd.read_excel(output_file)
else:
    existing_df = pd.DataFrame(columns=["id", "latitude", "longitude", "date"])

existing_df.set_index('id', inplace=True)

for _, row in df.iterrows():
    location_id = int(row['id'])
    lat = row['latitude']
    lon = row['longitude']

    print(f"✅ Date for ID: {location_id}")

    if location_id in existing_df.index and existing_df.loc[location_id, 'date'] not in ['Unknown', 'Error']:
        continue

    date = get_metadata(lat, lon)

    existing_df.loc[location_id] = [lat, lon, date]

existing_df.reset_index(inplace=True)
existing_df.to_excel(output_file, index=False)

wb = load_workbook(output_file)
ws = wb.active

alignment_center = Alignment(horizontal="center", vertical="center")

for row in ws.iter_rows():
    for cell in row:
        cell.alignment = alignment_center

wb.save(output_file)